In [ ]:
# finetune bert
# IEMOCAP text data : transcriptions_labels.csv
# tensorflow

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/emotion_recognition/transcriptions_labels.csv')

In [ ]:
df.head()

,TRANSCRIPTION,EMOTION
0,Excuse me.,neu
1,Yeah.,neu
2,Is there a problem?,neu
3,You did.,xxx
4,You were standing at the beginning and you di...,xxx


In [ ]:
df['EMOTION'].unique()

array(['neu', 'xxx', 'fru', 'ang', 'sad', 'hap', 'exc', 'sur', 'oth',
       'fea', 'dis'], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(df[['EMOTION']]).toarray())

df2 = df.join(encoder_df)

df2.head()

,TRANSCRIPTION,EMOTION,0,1,2,3,4,5,6,7,8,9,10
0,Excuse me.,neu,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Yeah.,neu,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Is there a problem?,neu,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,You did.,xxx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,You were standing at the beginning and you di...,xxx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2['TRANSCRIPTION'],encoder_df)
X_train.head(4)

651      So maybe we are in the wrong spot, but we are...
9265     Wow.  Well, that's really important, obviousl...
7419     For God's sake, three years.  Nobody comes ba...
8419                     So you're not going to leave me.
Name: TRANSCRIPTION, dtype: object

In [ ]:
y_train.head()

,0,1,2,3,4,5,6,7,8,9,10
651,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9265,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8419,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
436,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
!pip install -U "tensorflow==2.9.*"
!pip install -q -U "tensorflow-text==2.9.*"
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 26.8 MB/s eta 0:00:00


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(11, activation='softmax', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size = 32)

Epoch 1/100
236/236 [==============================] - 85s 310ms/step - loss: 1.9666 - accuracy: 0.2340
Epoch 2/100
236/236 [==============================] - 74s 312ms/step - loss: 1.8900 - accuracy: 0.2569
Epoch 3/100
236/236 [==============================] - 75s 317ms/step - loss: 1.8554 - accuracy: 0.2719
Epoch 4/100
236/236 [==============================] - 76s 323ms/step - loss: 1.8261 - accuracy: 0.2913
Epoch 5/100
236/236 [==============================] - 76s 323ms/step - loss: 1.8024 - accuracy: 0.3006
Epoch 6/100
236/236 [==============================] - 77s 325ms/step - loss: 1.7883 - accuracy: 0.3000
Epoch 7/100
236/236 [==============================] - 77s 326ms/step - loss: 1.7858 - accuracy: 0.3072
Epoch 8/100
236/236 [==============================] - 77s 325ms/step - loss: 1.7701 - accuracy: 0.3084
Epoch 9/100
236/236 [==============================] - 77s 326ms/step - loss: 1.7542 - accuracy: 0.3154
Epoch 10/100
236/236 [==============================] - 77s 327m

In [ ]:
y_predicted = model.predict(X_test)

79/79 [==============================] - 25s 310ms/step


In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy

In [ ]:
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(y_predicted, y_test)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=2510.0>

In [ ]:
m.result().numpy()

0.3752988

In [ ]:
y_train_predicted = model.predict(X_train)

236/236 [==============================] - 75s 318ms/step


In [ ]:
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(y_train_predicted, y_train)
m.result().numpy()

0.39394343